In [1]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

from dotenv import load_dotenv

load_dotenv()

def get_conversational_chain():

    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)


    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain



def user_input(user_question):
    embeddings = OpenAIEmbeddings()
    
    new_db = FAISS.load_local("../faiss_index", embeddings, allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)

    chain = get_conversational_chain()

    
    response = chain.invoke(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)

    print(response)
    print(len(new_db.docstore.__dict__['_dict']))
    print(docs)
    print(new_db.docstore.__dict__['_dict'])

In [2]:
user_input("prerequisites for COEN10?")

{'output_text': 'Prerequisites for COEN10 are not available in the provided context.'}
4
[Document(page_content='systems that process, organize, analyze large-scale information on the Web. Search engine technology, recommender systems, cloud computing, social network analysis. Prerequisite: AMTH 108 or MATH 122; CSEN/COEN 12 or CSCI 61; or permission of the instructor. (4 units) 171. Principles of Design and Implementation of Programming Languages High-level programming language concepts and constructs. Costs of use and implementation of the constructs. Issues and trade-offs in the design and implementation of programming languages. Critical look at several modern high-level programming languages. Prerequisite: a grade of C- or better in CSEN/COEN 79. (4 units) 174. Software Engineering Software development life cycle. Project teams, documentation, and group dynamics. Software cost estimation. Requirements engineering and design. Data modeling, object modeling, and object-oriented anal

In [6]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

prefix_template = '''
    You are a narrowly-scoped but highly effective AI agent that functions as a
    single step in a series of prompt transformation agents, working to create a chatbot system.

    Your only job is to rephrase the base prompt such that any course IDs within the prompt
    are put in a specific format. These course IDs generally come in the form of a four letter acronym
    indicating the department of the course, followed by a number indicating the specific course in that department.
    There may one or two characters following the number indicating special courses such as labs. Your only job is
    to replace any course IDs you identify in the base prompt with a correctly formatted version according to the
    following formatting rules:

    1. Any letters in the course ID should always be capitalized. This includes both the starting four letters
       indicating the course department as well as any trailing letters after the course number.
    2. There should always be an underscore (i.e. "_" ) character between the course department acronym and
       the course number. This means no spaces.
    3. If a course ID is in a condenced format as part of a list, separate them out into full IDs. 

    Examples are included below to help make this process clear. Remember, it is crucial that you do not alter
    the prompt in any way except to reformat these course IDs. Other than the correctly formatted course IDs
    the base prompt should be returned verbatim.

    {format_instructions}

'''

examples = [
    {
        "base_prompt": "What are the prereqs for coen10",
        "formatted_prompt": "What are the prereqs for COEN_10"
    },
    {
        "base_prompt": "Is ACTG 142L a corequisite with ACTG 142?",
        "formatted_prompt": "Is ACTG_142L a corequisite with ACTG_142?"
    },
    {
        "base_prompt": "Whst will I lear in psyc 134",
        "formatted_prompt": "Whst will I lear in PSYC_134"
    },
    {
        "base_prompt": "Are there any similar courses to RSOC 106 R offered next spring?",
        "formatted_prompt": "Are there any similar courses to RSOC_106R offered next spring?"
    },
    {
        "base_prompt": "Are elen 20 and 21 good courses to take if you want to work at NASA in 10 years",
        "formatted_prompt": "Are ELEN_20 and ELEN_21 good courses to take if you want to work at NASA in 10 years"
    }
]

prompt_template = "Base Prompt: {base_prompt}\nFormatted Prompt: {formatted_prompt}"


prompt_hydrate = PromptTemplate(
    input_variables=["base_prompt", "formatted_prompt"], template=prompt_template
)

class FormattedPrompt(BaseModel):
    question: str = Field(description="formatted prompt with correct course IDs")

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=FormattedPrompt)


final_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt_hydrate,
    prefix=prefix_template,
    suffix="Base Prompt: {input}",
    input_variables=["input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


# print(final_prompt.format(input="what ar ethe prereqs to COEN 12?"))

In [7]:
llm = ChatOpenAI(model_name='gpt-4', temperature=0.2)

chain = final_prompt | llm

# output = chain.invoke({
#     "input": "what ar ethe prereqs to COEN 12?"
# })

# print(output)

In [8]:
import getpass
import os
from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
import getpass
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool, QuerySQLCheckerTool
from langchain_core.runnables import RunnableLambda, RunnableParallel
from langchain.globals import set_debug
from langchain_core.tools import tool


load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini")

set_debug(False)

# if not os.environ.get("OPENAI_API_KEY"):
#     os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Comment out the below to opt-out of using LangSmith in this notebook. Not required.
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
    os.environ["LANGCHAIN_TRACING_V2"] = "true"

#init database
host = "scu-info.cpkec8qcwpzs.us-east-2.rds.amazonaws.com"
port = 5432  # Default PostgreSQL port
database = "SCU_INFO"
user = "sakadaai"
password = "sakadaai2000"
db_url = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(db_url)
db = SQLDatabase(engine)

print(db.dialect)
print(db.get_usable_table_names())

sql_prompt_template = '''

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
Your Output: SQL Query to run

Only use the following tables:
{table_info}

Question: {input}

Your Output:
'''

sql_prompt_hydrate = PromptTemplate(
    input_variables=["input", "table_info"], template=sql_prompt_template
)

@tool
def query_fixer(s: str) -> str:
    '''Properly reformats SQL Query'''

    if s.startswith("SQLQuery:"):
        s2 = s[10:]
        if "```sql" in s2:
                return s2[7:-3]
        else:
            return s2
    else:
        return s

fix_query = RunnableParallel(
    output=RunnableLambda(query_fixer)
)

write_query = create_sql_query_chain(llm, db)
# write_query = 
check_query = QuerySQLCheckerTool(llm=llm, db=db)
execute_query = QuerySQLDataBaseTool(db=db)
chain = final_prompt | llm | parser | write_query | query_fixer | execute_query

# | check_query

# chain.get_prompts()[0].pretty_print()
response = chain.invoke({"input": "How many graduate coen courses are there?"}) #need to use "question" as input dict key
print(response)

postgresql
['courses']
[(0,)]


In [67]:
x = """```sql
SELECT COUNT(*) AS "total_courses"
FROM courses
WHERE "academicunits" @> ARRAY['Graduate'] AND "coursesubjects" @> ARRAY['CSEN'];
```"""

print(x[7:-3])

SELECT COUNT(*) AS "total_courses"
FROM courses
WHERE "academicunits" @> ARRAY['Graduate'] AND "coursesubjects" @> ARRAY['CSEN'];



In [9]:
from pydantic import BaseModel
from langchain.schema import Document
from langchain_core.retrievers import BaseRetriever
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool, QuerySQLCheckerTool
from langchain.chains import create_sql_query_chain
from typing import List, Dict, Any, Optional
from langchain.callbacks.manager import CallbackManagerForRetrieverRun, AsyncCallbackManagerForRetrieverRun
from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
import os
import getpass
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
import time
from langchain.retrievers import EnsembleRetriever



# It's so beautiful but I fucking hate this thing
################################################################################################################
class SQLRetriever(BaseRetriever):
    llm: Any  # The language model used for generating SQL queries
    db: SQLDatabase  # The database instance for connecting and running SQL queries

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        """
        Synchronous method to retrieve relevant documents based on the query.
        
        Args:
        - query: String query to be converted into SQL.
        - run_manager: Callback manager to handle progress or other callback functionality.

        Returns:
        - List[Document]: List of LangChain Document objects.
        """
        # Create and execute the SQL query chain

        # write_query = create_sql_query_chain(llm, db)
        # check_query = QuerySQLCheckerTool(llm=llm, db=db)
        # execute_query = QuerySQLDataBaseTool(db=db)
        # query_chain = write_query | check_query | execute_query
        # response = query_chain.invoke({"question": query})

        response = chain.invoke({"input": query})
        
        # Convert SQL results into LangChain Document objects
        documents = self._format_sql_result(response)
        return documents

    async def _aget_relevant_documents(
        self, query: str, *, run_manager: AsyncCallbackManagerForRetrieverRun
    ) -> List[Document]:
        """
        Asynchronous method to retrieve relevant documents based on the query.
        
        Args:
        - query: String query to be converted into SQL.
        - run_manager: AsyncCallbackManager to handle async callbacks.

        Returns:
        - List[Document]: List of LangChain Document objects.
        """
        # Create and execute the SQL query chain
        write_query = create_sql_query_chain(llm, db)
        check_query = QuerySQLCheckerTool(llm=llm, db=db)
        execute_query = QuerySQLDataBaseTool(db=db)
        query_chain = write_query | check_query | execute_query
        response = await query_chain.invoke_async({"question": query})
        
        # Convert SQL results into LangChain Document objects
        documents = self._format_sql_result(response)
        return documents

    def _format_sql_result(self, result: Any) -> List[Document]:
        """
        Formats the SQL query result into LangChain Document objects.
        
        Args:
        - result: The raw result from the SQL query execution.

        Returns:
        - List[Document]: A list of LangChain Document objects, each containing 
          a row of the SQL result with metadata.
        """
        documents = []
        content = str(result)  # Convert the row into a string or format as needed
        metadata = {"source": "SQL Database"}  # Add any additional metadata as needed
        documents.append(Document(page_content=content, metadata=metadata))
        
        return documents
################################################################################################################




# Setup Openai
load_dotenv(find_dotenv()) 
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)

# Setup the database connection
host = "scu-info.cpkec8qcwpzs.us-east-2.rds.amazonaws.com"
port = 5432  # Default PostgreSQL port
database = "SCU_INFO"
user = "sakadaai"
password = "sakadaai2000"
db_url = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(db_url)
db = SQLDatabase(engine)

# Initialize custom retriever
sql_retriever = SQLRetriever(llm=llm, db=db)

# Test sql retrieval 
documents = sql_retriever._get_relevant_documents("What is the acla102Z course?", run_manager=None)
# for doc in documents:
#     print(doc.page_content)
# print(documents)



# IMPORTANT: BEFORE YOU TRY ANYTHING BELOW, MAKE SURE THE STUFF ABOVE WORKS FIRST
# This pinecone initialization is already in views.py
# I've changed to new pinecone gitbook/calendar-only db
##############################################################################################################

index_name = "calendar-gitbook-scu"
index_dimension = 3072
index_namespace = "scu"
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'


pc = Pinecone(api_key=PINECONE_API_KEY)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=index_dimension, 
        metric="cosine", 
        spec=ServerlessSpec(
            cloud=cloud, 
            region=region
        ) 
    )
while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)
index = pc.Index(index_name)

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = PineconeVectorStore(index=index, embedding=embeddings, namespace=index_namespace)
pinecone_retriever =  vectorstore.as_retriever(
    search_kwargs={"k": 10}
)
##############################################################################################################

ensemble_retriever = EnsembleRetriever(
    retrievers=[sql_retriever, pinecone_retriever], weights=[0.5, 0.5]
)

print(ensemble_retriever.invoke("what are prerequisites to csen11"))

config = {"configurable": {"search_kwargs_faiss": {"k": 3}}}
# print(ensemble_retriever.invoke("apples", config=config))

[Document(metadata={'source': 'SQL Database'}, page_content='[(["\'OMIS_30\'", "\'CSCI_10\'", "\'COEN_11\'"],)]'), Document(metadata={'source': 'gitbook_data\\chapter-3-college-of-arts-and-sciences\\mathematics-and-computer-science.html'}, page_content='10. Introduction to Computer Science\n\nIntroduction to computer science, and computer programming in Python. Basic programming structures, conditionals, loops, functions, recursion, arrays. Topics relating to the applications of and social impact of computing, including privacy, artificial intelligence, computation in physics, psychology, and biology. Discussion of cryptography, computation through history, networks, hardware, and basic runtime analysis. Includes weekly lab. CSCI 10 may be taken for credit if the student has received credit for CSEN/COEN 10, but not CSEN/COEN 11 or a similar introductory programming course, or CSCI 60. (5 units)\n\n60. Introduction to C++ and Object-Oriented Programming\n\nBasic object-oriented program

In [81]:
set_debug(True)